In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Classificação de textos com Generative AI na Vertex AI

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/language/intro_palm_api.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Execute no Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/language/intro_palm_api.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      Veja no GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/blob/main/language/intro_palm_api.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Execute no Vertex AI Workbench
    </a>
  </td>
</table>

## Visão geral

Modelos generativos como o PaLM 2 são modelos de linguagem poderosos usados para várias tarefas de processamento de linguagem natural (NLP). Uma delas é a classificação de texto, que envolve a atribuição de uma ou mais categorias a um determinado trecho de texto. Embora a classificação de texto possa ser feita usando técnicas NLP tradicionais, os LLMs podem realizar a classificação fornecendo prompts (em oposição a dados rotulados específicos do domínio), o que pode acelerar o tempo necessário para criar uma solução de classificação de texto. Os modelos de classificação baseados em LLMs podem ser ajustados com muitos exemplos por meio de treinamento de modelo personalizado, mas isso está além do escopo deste notebook.

Neste notebook, você explorará como fazer classificação de texto usando prompts com a API PaLM. Saiba mais sobre os prompts de classificação na [documentação oficial](https://cloud.google.com/vertex-ai/docs/generative-ai/text/classification-prompts).

### Objetivo

No final do notebook, você será capaz de usar um LLM para executar várias tarefas de classificação, incluindo:

* Classificação de texto com prompt zero-shot
* Classificação de texto com prompt few-shot
* Tarefas como:
     * Análise de sentimentos
     * Classificação do tópico
     * Detecção de spam
     * Reconhecimento de intenção
     * Identificação do idioma
     * Detecção de toxicidade
     * Detecção de emoções

### Custos
Este tutorial usa os seguintes componentes de Google Cloud:

* Vertex AI Generative AI Studio

Saiba mais sobre possíveis custos envolvidos [preços da Vertex AI](https://cloud.google.com/vertex-ai/pricing),
e use a [Calculadora de preços](https://cloud.google.com/products/calculator/)
para gerar uma estimativa de custo com base no uso projetado.

## Primeiros Passos

### Instalando os SDK da Vertex AI e da Cloud Translate API

In [ ]:
!pip install google-cloud-aiplatform google-cloud-translate --upgrade --user

**Somente Colab:** Descomente a célula a seguir para reiniciar o kernel ou use o botão para reiniciar o kernel.

In [ ]:
# # Reinicia automaticamente o kernel após as instalações para que seu ambiente possa acessar os novos pacotes
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

### Autenticando seu ambiente de notebook
* Se você estiver usando o **Colab** para executar este notebook, descomente a célula abaixo e continue.
* Se você estiver usando o **Vertex AI Workbench**, confira as instruções de configuração [aqui](../../setup-env/README.md).

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

### Importando as bibliotecas necessárias

**Somente Colab:** Descomente a célula a seguir para realizar o processo adequado de inicialização da SDK da Vertex AI.  

In [ ]:
# import vertexai

# PROJECT_ID = "[seu-project-id]"  # @param {type:"string"}
# vertexai.init(project=PROJECT_ID, location="us-central1")

In [ ]:
import pandas as pd
from vertexai.preview.language_models import TextGenerationModel

#### Carregando o modelo `text-bison`

In [ ]:
generation_model = TextGenerationModel.from_pretrained("text-bison@001")

#### Criando a função *wrapper* para utilizar os modelos em Português

Até o momento desde treinamento, as API do Generative AI Studio suportam somente interações no idioma inglês. Para fazermos interações utilizando o idioma português, vamos utilizar a [Cloud Translation API](https://cloud.google.com/translate) para traduzir as nossas solicitações do português para o inglês e as respostas da API de inglês para português.

In [ ]:
from google.cloud import translate

project_id = !gcloud config list project
project_id = project_id[1].split('=')[1].strip()
parent = f'projects/' + project_id


def traduza(texto, idioma_destino):
    client = translate.TranslationServiceClient()

    response = client.translate_text(
        parent=parent,
        contents=[texto],
        target_language_code=idioma_destino,
        mime_type="text/plain"
    )

    return response.translations[0].translated_text

## Classificação de texto

Na seção abaixo, você explorará prompts zero-shot, few-shot e alguns tipos comuns de tarefas de classificação de texto.

### Prompt zero-shot

O prompt zero-shot é onde você não fornece exemplos com rótulos e conta com o LLM para fazer a classificação por conta própria.

In [ ]:
prompt = traduza("""
Ordene el texto a continuación:\n
texto: "Hoy vi un animal peludo en el parque - Tenía una cola larga y ojos grandes".
clase: perros, gatos
""", "en")

print(
    traduza(generation_model.predict(prompt=prompt, max_output_tokens=256, temperature=0.1).text, "es")
)

### Prompt few-shot

Com prompt few-shot, você fornece exemplos para o modelo PaLM e espera que ele faça a classificação com base nos exemplos fornecidos.

In [ ]:
prompt = traduza("""
¿Cuál es el tema de un titular de noticias en particular? \n
- negocio \n
- entretenimiento \n
- salud \n
- deportes \n
- tecnología \n\n

Texto: Revisión práctica de expertos del Pixel 7 Pro. \n
La respuesta es: tecnología \n

Texto: ¿Dejar de fumar? \norte
La respuesta es: salud \n

Texto: ¿Pajaritos o cucos? Los 5 mejores consejos para golpear por debajo del par \n
La respuesta es: deportes \n

Texto: El alivio del aumento del salario mínimo local parece más remoto\n
La respuesta es: negocio \n

Texto: No adivinarán quién acaba de llegar a Bari, Italia, para el estreno de la película. \n
La respuesta es:
""", "en")

print(
    traduza(generation_model.predict(prompt=prompt, max_output_tokens=256, temperature=0.1).text, "es")
)

### Outros exemplos de classificação

Explore alguns prompts de classificação de texto mais comuns abaixo, todos baseados em prompts zero-shot. Você também pode transformar alguns deles em prompts few-shot, fornecendo seus próprios exemplos personalizados de texto e as classes de saída associadas.

#### Classificação de tópico

In [ ]:
prompt = traduza("""
Clasifique un fragmento de texto en uno de varios temas predefinidos, como deportes, política o entretenimiento. \n
texto: El presidente Biden visitará la India en el mes de marzo para discutir algunas oportunidades. \n
clase:
""", "en")

print(
    traduza(generation_model.predict(prompt=prompt, max_output_tokens=256, temperature=0.1).text, "es")
)

####  Detecção de spam

In [ ]:
prompt = traduza("""
Dado un correo electrónico, clasificarlo como spam o no spam. \norte
correo electrónico: hola usuario,\n
        ha sido seleccionado como ganador de la lotería y puede ganar hasta 1 millón de dólares. \norte
        comparta amablemente sus datos bancarios y podemos proceder desde allí. \n\n

        de, \n
        Departamento oficial de lotería de EE. UU.
""", "en")

print(
    generation_model.predict(prompt=prompt, max_output_tokens=256, temperature=0.1).text
)

#### Reconhecimento de intenção

In [ ]:
prompt = traduza("""
Dada la entrada de un usuario, clasifique su intención, como "buscar información", "hacer una reserva" o "hacer un pedido". \norte
Entrada del usuario: Hola, ¿puedes reservar una mesa para dos en Google Restaurant para el 1 de mayo?
""", "en")

print(
    traduza(generation_model.predict(prompt=prompt, max_output_tokens=256, temperature=0.1).text, "es")
)

#### Identificação de linguagem

In [ ]:
prompt = traduza("""
Dado un fragmento de texto, clasificar el idioma en el que está escrito. \n
texto: Selam nasıl gidiyor?
idioma:
""", "en")

print(
    traduza(generation_model.predict(prompt=prompt, max_output_tokens=256, temperature=0.1).text, "es")
)

#### Detecção de toxicidade

In [ ]:
prompt = traduza("""
Dado un fragmento de texto, clasifícalo como tóxico o no tóxico. \n
texto: Intenta que puedas.
""", "en")

print(
    traduza(generation_model.predict(prompt=prompt, max_output_tokens=256, temperature=0.1).text, "es")
)

#### Detecção de emoção

In [ ]:
prompt = traduza("""
Dado un fragmento de texto, califique la emoción que transmite, como "felicidad" o "ira". \n
texto: estoy muy feliz por la noticia de ayer
""", "en")

print(
    traduza(generation_model.predict(prompt=prompt, max_output_tokens=256, temperature=0.5).text, "es")
)

### Avaliação

Você pode avaliar as saídas da tarefa de classificação de texto se as classes de informações básicas estiverem disponíveis. Para mostrar como isso funciona, comece criando um dataframe simples com análises de produtos e o sentimento de verdade.

In [ ]:
review_data = {
    "review": [
         "Me encanta este producto. ¡Tiene todo lo que busco!",
         "Todo lo que puedo decir es que estará feliz después de comprar este producto",
         "es demasiado caro y no vale la pena el precio",
         "Él está bien. Ni bien ni tan mal",
    ],
    "sentiment_groundtruth": ["positivo", "positivo", "negativo", "neutral"],
}

review_data_df = pd.DataFrame(review_data)
review_data_df

Agora que você tem os dados com avaliações e sentimentos como rótulos de informações básicas, pode chamar o modelo de geração de texto para cada linha de avaliação usando a função `apply`. Cada linha usará o prompt na coluna `review` para prever o sentimento usando a API PaLM e armazenará os resultados na coluna `sentiment_prediction`.

In [ ]:
def get_sentiment(row):
    prompt = traduza(f"""Califique la opinión de las reseñas a continuación como "positiva", "neutral" o "negativa". \n\n
                 revisión: {row} \n
                 sentimiento:
               """, "en")
    response = generation_model.predict(prompt=prompt).text
    return traduza(response, "es")


review_data_df["sentiment_prediction"] = review_data_df["review"].apply(get_sentiment)
review_data_df

No final, você pode chamar a função `classification_report` do móulo `scikit-learn` para medir a precisão e outras métricas de classificação, passando sentimentos de verdade absoluta `sentiment_groundtruth` e sentimento previsto `sentiment_prediction`:

In [ ]:
from sklearn.metrics import classification_report

print(
    classification_report(
        review_data_df["sentiment_groundtruth"], review_data_df["sentiment_prediction"]
    )
)